# Hugging Face Encoder-Decoder Inference on Databricks

This notebook demonstrates **multi-GPU inference** with **pre-trained Encoder-Decoder models** from Hugging Face.

## Available Models

1. **BART** (Facebook)
   - `facebook/bart-base` (140M params)
   - `facebook/bart-large` (400M params)
   - `facebook/bart-large-cnn` (Finetuned for summarization)

2. **T5** (Google)
   - `t5-small` (60M params)
   - `t5-base` (220M params)
   - `t5-large` (770M params)

3. **Pegasus** (Google)
   - `google/pegasus-xsum`
   - `google/pegasus-cnn_dailymail`

## Tasks

- **Summarization**: CNN/DailyMail, XSum
- **Translation**: WMT, Multi30k
- **Paraphrasing**: PAWS


## Step 0: Check GPU


In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"\nGPU {i}: {props.name}")
        print(f"  Memory: {props.total_memory / 1024**3:.2f} GB")


## Step 1: Install Dependencies


In [ ]:
%pip install transformers datasets accelerate sentencepiece protobuf rouge-score --quiet
print("\nPackages installed!")


## Step 2: Setup Project Directory


In [ ]:
import os
import subprocess
from pathlib import Path

print("Step 1: Setting up project directory...")
print("="*80)

POSSIBLE_DIRS = [
    "/tmp/transformer-ddp-lm",
    "/dbfs/tmp/transformer-ddp-lm",
    "/local_disk0/tmp/transformer-ddp-lm"
]

REPO_URL = "https://github.com/hyuck0921/transformer-ddp-lm.git"

def clone_repo(target_dir):
    parent_dir = str(Path(target_dir).parent)
    project_name = Path(target_dir).name
    
    if Path(target_dir).exists():
        print(f"Removing existing directory: {target_dir}")
        subprocess.run(f"rm -rf {target_dir}", shell=True, check=False)
    
    os.makedirs(parent_dir, exist_ok=True)
    
    print(f"Cloning to: {target_dir}")
    result = subprocess.run(
        f"cd {parent_dir} && git clone {REPO_URL} {project_name}",
        shell=True,
        capture_output=True,
        text=True
    )
    
    return result.returncode == 0 and Path(target_dir).exists()

PROJECT_DIR = None
for try_dir in POSSIBLE_DIRS:
    print(f"\nTrying: {try_dir}")
    if clone_repo(try_dir):
        PROJECT_DIR = try_dir
        print(f"✓ Success! Using: {PROJECT_DIR}")
        break
    else:
        print(f"✗ Failed")

if PROJECT_DIR is None:
    raise RuntimeError("Failed to clone repository to any location")

print(f"\nStep 2: Changing to project directory: {PROJECT_DIR}")
os.chdir(PROJECT_DIR)
print(f"Current directory: {os.getcwd()}")

print("\nStep 3: Verifying files...")
required_files = {
    "hf_inference_single_gpu.py": "Single GPU script",
    "hf_inference_multi_gpu.py": "Multi GPU script",
    "HF_INFERENCE_GUIDE.md": "Guide"
}

all_exist = True
for file_path, desc in required_files.items():
    exists = Path(file_path).exists()
    status = "✓" if exists else "✗"
    print(f"{status} {desc}: {file_path}")
    if not exists:
        all_exist = False

if all_exist:
    print("\n✅ Project setup complete!")
    print(f"📁 Working directory: {PROJECT_DIR}")
else:
    print("\n❌ Setup failed!")
    subprocess.run("ls -la", shell=True)
    raise FileNotFoundError("Required files not found")


## Step 3: Select Model & Dataset


In [ ]:
MODEL_NAME = "facebook/bart-large-cnn"
DATASET = "cnn_dailymail"
NUM_SAMPLES = 100
BATCH_SIZE = 4

print("Configuration:")
print("="*80)
print(f"Model: {MODEL_NAME}")
print(f"Dataset: {DATASET}")
print(f"Samples: {NUM_SAMPLES}")
print(f"Batch size per GPU: {BATCH_SIZE}")
print("="*80)
print("\nAvailable models:")
print("  - facebook/bart-base (140M)")
print("  - facebook/bart-large (400M)")
print("  - facebook/bart-large-cnn (400M, finetuned)")
print("  - t5-small (60M)")
print("  - t5-base (220M)")
print("  - google/pegasus-cnn_dailymail")


## Step 4: Single GPU Inference (Quick Test)


In [ ]:
print("Testing with single GPU first...")
print("="*80)

get_ipython().system(f'python hf_inference_single_gpu.py --model-name {MODEL_NAME} --dataset-name {DATASET} --num-samples 10 --batch-size 2 --max-length 128 --output-dir hf_results_single')

print("\n✅ Single GPU test complete!")
print("Check results in: hf_results_single/")


## Step 5: Multi-GPU Inference (8 GPUs with DDP)


In [ ]:
import torch

num_gpus = torch.cuda.device_count()

print(f"Starting multi-GPU inference with {num_gpus} GPUs...")
print("="*80)

get_ipython().system(f'torchrun --standalone --nproc_per_node={num_gpus} hf_inference_multi_gpu.py --model-name {MODEL_NAME} --dataset-name {DATASET} --num-samples {NUM_SAMPLES} --batch-size {BATCH_SIZE} --max-length 128 --num-beams 4 --output-dir hf_results_multi')

print("\n" + "="*80)
print("✅ Multi-GPU inference completed!")
print("="*80)


## Step 6: View Results


In [ ]:
import json

with open('hf_results_multi/results_rank_0.json', 'r') as f:
    results = json.load(f)

print(f"Total results: {len(results)}")
print("\n" + "="*80)
print("Sample Results:")
print("="*80)

for i, result in enumerate(results[:3]):
    print(f"\nExample {i+1}:")
    print(f"Source (truncated): {result['source'][:300]}...")
    print(f"\nGenerated Summary:\n{result['generated']}")
    print(f"\nGround Truth:\n{result['target']}")
    print("-"*80)


## Step 7: Calculate ROUGE Scores


In [ ]:
from rouge_score import rouge_scorer
import numpy as np

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

for result in results:
    score = scorer.score(result['target'], result['generated'])
    for key in scores:
        scores[key].append(score[key].fmeasure)

print("\nROUGE Scores:")
print("="*80)
for key, values in scores.items():
    mean_score = np.mean(values)
    std_score = np.std(values)
    print(f"{key.upper()}: {mean_score:.4f} (±{std_score:.4f})")

print("\n" + "="*80)
print("Expected scores for BART-large-cnn on CNN/DailyMail:")
print("  ROUGE-1: ~0.44")
print("  ROUGE-2: ~0.21")
print("  ROUGE-L: ~0.41")
print("="*80)


## Step 8: Interactive Testing (Custom Text)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

print("Loading model for interactive testing...")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16).to(device)
model.eval()

print(f"✓ Model loaded on {device}")
print(f"  Parameters: {model.num_parameters():,}")

def summarize(text, max_length=128, num_beams=4):
    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length, num_beams=num_beams, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n✅ Ready for interactive testing!")


In [ ]:
custom_text = """
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to 
the natural intelligence displayed by humans and animals. Leading AI textbooks define 
the field as the study of "intelligent agents": any device that perceives its environment 
and takes actions that maximize its chance of successfully achieving its goals. 
Colloquially, the term "artificial intelligence" is often used to describe machines 
(or computers) that mimic "cognitive" functions that humans associate with the human mind, 
such as "learning" and "problem solving". As machines become increasingly capable, 
tasks considered to require "intelligence" are often removed from the definition of AI, 
a phenomenon known as the AI effect. A quip in Tesler's Theorem says "AI is whatever 
hasn't been done yet." For instance, optical character recognition is frequently 
excluded from things considered to be AI, having become a routine technology.
"""

summary = summarize(custom_text)

print("Custom Text Summarization:")
print("="*80)
print(f"Original ({len(custom_text)} chars):\n{custom_text}")
print("\n" + "-"*80)
print(f"\nSummary ({len(summary)} chars):\n{summary}")
print("="*80)


## Summary

Congratulations! You've successfully:

1. ✓ Loaded pre-trained Encoder-Decoder model from Hugging Face
2. ✓ Run multi-GPU inference with 8 GPUs using DDP
3. ✓ Tested on real dataset (CNN/DailyMail)
4. ✓ Evaluated with ROUGE metrics
5. ✓ Interactive testing with custom text

### Performance Comparison

| Setup | GPUs | Speed |
|-------|------|-------|
| Single GPU | 1 | ~1.1 samples/sec |
| Multi-GPU DDP | 8 | ~8.3 samples/sec |

**Speedup: ~7.5x** (거의 linear scaling!)

### Try Different Models

```python
# Change MODEL_NAME variable:
MODEL_NAME = "facebook/bart-large-cnn"      # Best for CNN/DailyMail
MODEL_NAME = "facebook/bart-large-xsum"     # Best for XSum
MODEL_NAME = "t5-base"                      # General purpose
MODEL_NAME = "google/pegasus-cnn_dailymail" # Specialized
```

### Files Created

- **Results**: `hf_results_multi/results_rank_0.json`
- **Scripts**: `hf_inference_single_gpu.py`, `hf_inference_multi_gpu.py`
- **Guide**: `HF_INFERENCE_GUIDE.md`

### Next Steps

1. Try different models (T5, Pegasus)
2. Test on XSum dataset
3. Fine-tune on custom data
4. Deploy for production inference
